In [ ]:
!pip install -U ultralytics

from ultralytics import YOLO
import os
import glob
from tqdm.notebook import tqdm

# --- 設定エリア ---
# 画像が入っているフォルダ
IMAGE_DIR = "/content/dataset_raw"
# ラベルを保存するフォルダ
OUTPUT_DIR = "/content/dataset_raw/labels"

# ★探したいものをリストで書く
CLASSES = ["sleeping dog", "awake dog"]

# 感度
CONF_THRESHOLD = 0.1

# ------------------

# 保存フォルダの作成
os.makedirs(OUTPUT_DIR, exist_ok=True)

# YOLO-Worldモデルのロード
print("モデルをロード中...（初回はダウンロードに時間がかかります）")
model = YOLO('yolov8x-worldv2.pt')

# クラスの設定
model.set_classes(CLASSES)

# 画像リストの取得
image_paths = glob.glob(os.path.join(IMAGE_DIR, "*.png"))
print(f"処理開始: {len(image_paths)}枚の画像を解析します...")

# 推論と保存実行
for img_path in tqdm(image_paths):
    # 推論
    results = model.predict(img_path, conf=CONF_THRESHOLD, verbose=False)

    filename = os.path.basename(img_path)
    txt_name = os.path.splitext(filename)[0] + ".txt"
    save_path = os.path.join(OUTPUT_DIR, txt_name)

    # 結果を書き込み
    content = []
    for result in results:
        for box in result.boxes:
            cls_id = int(box.cls[0])
            xywh = box.xywhn[0].tolist()

            line = f"{cls_id} {xywh[0]:.6f} {xywh[1]:.6f} {xywh[2]:.6f} {xywh[3]:.6f}"
            content.append(line)

    # ファイルに保存
    with open(save_path, "w") as f:
        if content:
            f.write("\n".join(content))

print("✅ アノテーション完了！")

In [ ]:
# --- 1. データセットの解凍 ---
# dataset.zip を /content/ に解凍します
!unzip -q dataset.zip -d /content/

# --- 2. YOLOv8のインストール ---
!pip install ultralytics

# --- 3. 学習の実行 ---
from ultralytics import YOLO

# ★重要: ラズパイ5用に「最軽量モデル (Nano)」を使用
model = YOLO('yolov8n.pt')

# 学習開始
# data: 設定ファイルの場所
# epochs: 学習回数 (まずは50回で様子見)
# imgsz: 画像サイズ (640が標準)
print("🚀 学習を開始します...")
model.train(data='/content/dataset/data.yaml', epochs=50, imgsz=640)

In [ ]:
import os
from google.colab import files

# 1. "best.pt" という名前のファイルを全検索
print("探しています...")
found_files = []
for root, dirs, filenames in os.walk('/content'):
    for filename in filenames:
        if filename == 'best.pt':
            full_path = os.path.join(root, filename)
            found_files.append(full_path)

# 2. 結果の表示とダウンロード
if found_files:
    print(f"見つかりました！: {found_files}")
    # 最新のもの（リストの最後）をダウンロード
    target_file = found_files[-1]
    print(f"ダウンロードを開始します: {target_file}")
    files.download(target_file)
else:
    print("❌ 見つかりませんでした。学習が完了していないか、エラーが起きている可能性があります。")

In [ ]:
from ultralytics import YOLO
import glob
import random
import cv2
import matplotlib.pyplot as plt
import os

# 1. モデルの読み込み（最新の学習結果を使用）
runs_dir = '/content/runs/detect'
latest_train = max(glob.glob(os.path.join(runs_dir, 'train*')), key=os.path.getmtime)
best_model_path = os.path.join(latest_train, 'weights', 'best.pt')

print(f"モデル読み込み中: {best_model_path}")
model = YOLO(best_model_path)

# 2. 検証用画像のリスト取得
# datasetフォルダの構成に合わせて拡張子を指定 (.png または .jpg)
valid_images = glob.glob('/content/dataset/valid/images/*.*')

if not valid_images:
    print("エラー: 検証用画像が見つかりません。パスを確認してください。")
else:
    # 3. ランダムに3枚選んで推論・表示
    print(f"検証画像数: {len(valid_images)}枚から、3枚をランダムテストします...")

    # 画像サイズを調整して表示する設定
    plt.figure(figsize=(15, 5))

    picked_images = random.sample(valid_images, min(3, len(valid_images)))

    for i, img_path in enumerate(picked_images):
        # 推論実行
        results = model.predict(img_path, conf=0.5) # 確信度0.5以上を表示

        # 結果の描画（YOLOの機能で枠付き画像を作成）
        result_array = results[0].plot()

        # Matplotlibで表示するために色を変換 (BGR -> RGB)
        result_rgb = cv2.cvtColor(result_array, cv2.COLOR_BGR2RGB)

        # 横に並べて表示
        plt.subplot(1, 3, i+1)
        plt.imshow(result_rgb)
        plt.axis('off')
        plt.title(f"Test {i+1}")

    plt.show()

In [ ]:
# --- 1. データセットの解凍 ---
# dataset.zip を /content/ に解凍します
!unzip -q dataset.zip -d /content/

# --- 2. YOLOv8のインストール ---
!pip install ultralytics

# --- 3. 学習の実行 ---
from ultralytics import YOLO

# ★重要: nanoモデルとの比較用に「small」を使用
model = YOLO('yolov8s.pt')

# 学習開始
# data: 設定ファイルの場所
# epochs: 学習回数 (まずは50回で様子見)
# imgsz: 画像サイズ (640が標準)
print("🚀 学習を開始します...")
model.train(data='/content/dataset/data.yaml', epochs=50, imgsz=640)

In [ ]:
import os
import datetime
from google.colab import files

print("🔎 すべての 'best.pt' を探しています...\n")

found_files = []

# runsフォルダ以下を全探索
for root, dirs, filenames in os.walk('/content/runs'):
    for filename in filenames:
        if filename == 'best.pt':
            full_path = os.path.join(root, filename)

            # ファイル情報の取得
            size_mb = os.path.getsize(full_path) / (1024 * 1024) # MB単位
            mtime = os.path.getmtime(full_path)
            dt = datetime.datetime.fromtimestamp(mtime)

            found_files.append({
                "path": full_path,
                "size": size_mb,
                "time": dt
            })

# 見つかったファイルを整理して表示
if found_files:
    # 新しい順に並べ替え
    found_files.sort(key=lambda x: x["time"], reverse=True)

    print(f"{'フォルダ名':<25} | {'サイズ':<10} | {'作成日時'}")
    print("-" * 60)

    for i, f in enumerate(found_files):
        folder_name = os.path.basename(os.path.dirname(os.path.dirname(f["path"]))) # train, train2...
        size_str = f"{f['size']:.1f} MB"
        time_str = f["time"].strftime('%H:%M:%S')

        print(f"{folder_name:<25} | {size_str:<10} | {time_str}")

    print("-" * 60)

    # 一番新しい（リストの先頭）ファイルを自動選択
    target = found_files[0]
    print(f"\n🚀 最新のモデル（{target['size']:.1f} MB）をダウンロードします: {target['path']}")
    files.download(target['path'])

else:
    print("❌ 'best.pt' が見つかりませんでした。学習エラーの可能性があります。")

In [ ]:
from ultralytics import YOLO
import glob
import random
import cv2
import matplotlib.pyplot as plt
import os

# 1. モデルの読み込み（最新の学習結果を使用）
runs_dir = '/content/runs/detect'
latest_train = max(glob.glob(os.path.join(runs_dir, 'train2')), key=os.path.getmtime)
best_model_path = os.path.join(latest_train, 'weights', 'best.pt')

print(f"モデル読み込み中: {best_model_path}")
model = YOLO(best_model_path)

# 2. 検証用画像のリスト取得
# datasetフォルダの構成に合わせて拡張子を指定 (.png または .jpg)
valid_images = glob.glob('/content/dataset/valid/images/*.*')

if not valid_images:
    print("エラー: 検証用画像が見つかりません。パスを確認してください。")
else:
    # 3. ランダムに3枚選んで推論・表示
    print(f"検証画像数: {len(valid_images)}枚から、3枚をランダムテストします...")

    # 画像サイズを調整して表示する設定
    plt.figure(figsize=(15, 5))

    picked_images = random.sample(valid_images, min(3, len(valid_images)))

    for i, img_path in enumerate(picked_images):
        # 推論実行
        results = model.predict(img_path, conf=0.5) # 確信度0.5以上を表示

        # 結果の描画（YOLOの機能で枠付き画像を作成）
        result_array = results[0].plot()

        # Matplotlibで表示するために色を変換 (BGR -> RGB)
        result_rgb = cv2.cvtColor(result_array, cv2.COLOR_BGR2RGB)

        # 横に並べて表示
        plt.subplot(1, 3, i+1)
        plt.imshow(result_rgb)
        plt.axis('off')
        plt.title(f"Test {i+1}")

    plt.show()